### Zarządzanie kontekstem. Bloki `with`

Obiekty menedżerów kontekstu istnieją w celu sterowania instrukcją `with` tak samo, jak iteratory istnieją do sterowania instrukcją `for`. Intrukcja `with` została zaprojektowana w celu uproszczenia wzorca `try/finally` i gwarantuje, że jakaś operacja zostanie wykonana nawet jeśli dany blok kodu zostanie przerwany. 

In [4]:
# Najbardziej typowy przykład użycia menedżera kontekstu to otwieranie plików.
with open("file.txt", "r", encoding="utf-8") as f:
    data = f.read()

In [5]:
# Zmienna f jest wciąż dostępna. Bloki `with` nie definiują nowego zasięgu.
f

<_io.TextIOWrapper name='file.txt' mode='r' encoding='utf-8'>

In [6]:
f.closed, f.encoding

(True, 'utf-8')

In [7]:
# Po wyjściu z bloku `with` plik jest zamykany.
f.read()

ValueError: I/O operation on closed file.

Obiekt menedżera kontekstu jest wynikiem wyliczenia wyrażenia następującego po `with`, ale wartość przywiązywana do zmiennej docelowej (w klauzuli `as`) jest wynikiem wywołania metody `__enter__` obiektu menedżera kontekstu. Metoda `__enter__` może zwrócić inny obiekt zamiast menedżera kontekstu. Gdy przepływ sterowania wyjdzie z bloku `with`, wywoływna jest metoda `__exit__` obiektu menedżera kontekstu, a nie obiektu zwróconego przez `__enter__`. Klauzula `as` jest opcjonalna, czasami menedżer kontekstu zwraca `None`, ponieważ nie ma żadnego przydatnego obiektu do zwrócenia użytkownikowi.

In [8]:
import sys

class LookingGlass:
    # Metoda __enter__ przyjuje tylko jeden argument - self.
    def __enter__(self):
        self.original_write = sys.stdout.write
        # Małpie łatanie metody write.
        sys.stdout.write = self.reverse_write # type: ignore
        # Zwracamy obiekt, który będzie przypisany do zmiennej po klauzuli `as`.
        return "JABBA"
    
    # Python wywołuje __exit__ z None, None, None jeśli wszystko poszło ok.
    # Jeśli wystąpił wyjątek, to są przekazywane jego typ, wartość i traceback.
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print("Do not divide by zero")
            return True # Wyjątek został obsłużony.
        # Jeśli __exit__ zwróci None lub cokolwiek innego, to wyjątek zostanie 
        # przekazany dalej.
        
    def reverse_write(self, text):
        self.original_write(text[::-1])

In [9]:
with LookingGlass() as what:
    print("Alice, Kitty and Snowdrop")
    print(what)

pordwonS dna yttiK ,ecilA
ABBAJ


In [10]:
print("Everything is normal now")

Everything is normal now


In [11]:
# Od wersji 3.10 można użyć `with` z wieloma menedżerami kontekstu
# zamiast je zagnieżdżać
with (
    LookingGlass() as example1,
    LookingGlass() as example2,
    LookingGlass() as example3
): ...

### Narzędzia `contextlib`

Dekorator `@contextmanager` jest narzędziem, które zbier trzy rozłączne funkcjonalności: dekorator, generator oraz instrukcję `with`. Zamiast pisać całą klasę menedżera kontekstu, implementujemy po prostu generator z instrukcją `yield`, która powinna produkować to co chcemy zwrócić z metody `__enter__`.

In [15]:
import contextlib
import sys


# Dekorator contextlib.contextmanager opakowuje funkcję w klasę,
# która implementuje metody __enter__ i __exit__.
@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write  # type: ignore
    msg = ""
    try:
        # Wszystko co jest przed yield jest odpowiednikiem __enter__.
        yield "JABBA" # Wartość zwracana przez yield będzie powiązana z klauzulą `as`.
        # Wszystko co jest po yield jest odpowiednikiem __exit__.
    except:
        msg = "Do not divide by zero"
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [16]:
with looking_glass() as what:
    print("Alice, Kitty and Snowdrop")
    print(what)

pordwonS dna yttiK ,ecilA
ABBAJ


In [17]:
what

'JABBA'

Dekorator `@contextlib.contextmanager` opakowuje funkcję w klasę która implementuje metody `__enter__` i `__exit__`. 

Metoda `__enter__` tej klasy:
- Wywołuje funkcję generatora i utrzymuje obiekt generatora - np. gen.
- Wywołuje `next(gen)` aby wykonać kod aż do słowa kluczowego `yield`.
- Zwraca wartość wyprodukowaną przez `next(gen)` aby można było ją powiązać ze zmienną po `as`.

Po zakończeniu działa bloku `with`, metoda `__exit__`:
- Sprawdza czy został przekazany wyjątek `exc_type`. Jeśli tak to wywoływane jest `gen.throw(exception)`, co powoduje, że wyjątek jest zgłaszany w wierszu `yield`.
- W przeciwnym razie wywoływane jest `next(gen)`, co wznawia wykonywanie funkcji generatora po instrukcji `yield`.

Objęcie instrukcji `yield` konstrukcją `try/finally` jest ceną wykorzystania `@contextmanager`, ponieważ nigdy nie wiemy co użytkownik menedżera kontekstu zrobi w bloku `with`.

In [20]:
# Mało znaną cecha dekoratora contextlib.contextmanager jest to,
# że dekorowane nim generatory same mogą zostać użyte jako dekoratory.
# Działa to, ponieważ contextlib.contextmanager jest implementowany
# w ramach klasy contextlib.ContextDecorator.
@looking_glass()
def reverse(text):
    print(text)
    
reverse("Hello")
print("Hello")

olleH
Hello


### Bloki `else` poza instrukcją `if`

Klauzula `else` może być używana nie tylko w instrukcjach `if` ale również w instrukcjach `for`, `while` oraz `try`. Semantyka `for/else`, `while/else` oraz `try/else` jest blisko powiązana ale bardzo różna od `if/else`. Reguły są natępujące:

- `for` - Blok `else` zadziała tylko wtedy, gdy pętla dotrze do końca. Nie zadziała jeśli opuścimy pętlę przy pomocy instrukcji `break`.
- `while` - Blok `else` zadziała tylko wtedy, gdy pętla zakończy się (warunek pętli stanie się falsy). Nie zadziała jeśli opuścimy pętlę przy pomocy instrukcji `break`.
- `try` - Blok `else` zadziała tylko wtedy, gdy żaden wyjątek nie zostanie zgłoszony w bloku `try`. 
  
We wszystkich przypadkach klauzula `else` jest też pomijana, jeśli wyjątek albo instrukcja `return`, `break` lub `continue` spowodują wyjście z głównego bloku instrukcji.

Słowo kluczowe `else` jest prawdopodobnie źle dobranym słowem kluczowym w powyższych przypadkach, ponieważ opisane bloki działają w trybie "zrób to a potem tamto". Dlatego też lepiej pasowałoby prawdopodobnie słowo `then`.

In [26]:
fruits = ("apple", "banana", "cherry")

# Typowy przykład użycia klauzuli `else` dla `for`.
for fruit in fruits:
    if fruit == "pineapple":
        break
else:
    raise ValueError("No pineapple found")

ValueError: No pineapple found

In [33]:
try:
    a = float("1")
except ValueError:
    print("Could not convert to float")
else:
    # Kod w bloku `else` jest wykonywany tylko jeśli nie wystąpił wyjątek.
    print("Conversion successful")
finally:
    print("Cleaning up")

Conversion successful
Cleaning up
